# To Do List 5.31.2023
- Import Data
- Split off Train, Test, Val set
- Inspect all combinations of inputs (matching = [s for s in pdf.columns if "x10" in s]) - Lecture 31
- Hyper tune (pipeline these were possible)
- Apply SMOTE
- 

Load data
Split off Val set
Apply SMOTE
Hyper tune
Pipeline those auto tuned parameters into new model
break off data into test and train
Run model and test on test and val set using tuned parameters
Evaluate

# Import the necessary libraries:

In [64]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import itertools
import statsmodels.api as sm

# Load & Explore the dataset:

In [5]:
data = pd.read_csv("Raw_Data/Churn_Data.csv")

In [6]:
data

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


In [8]:
data['area code'].value_counts().head(100)

415    1655
510     840
408     838
Name: area code, dtype: int64

In [9]:
data = data.drop(columns=['phone number','area code'] )

In [10]:
print(data.columns)

Index(['state', 'account length', 'international plan', 'voice mail plan',
       'number vmail messages', 'total day minutes', 'total day calls',
       'total day charge', 'total eve minutes', 'total eve calls',
       'total eve charge', 'total night minutes', 'total night calls',
       'total night charge', 'total intl minutes', 'total intl calls',
       'total intl charge', 'customer service calls', 'churn'],
      dtype='object')


In [11]:
data

,state,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,no,yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,no,no,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,no,no,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,yes,no,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


In [14]:
# Apply pd.get_dummies() to the desired columns
data = pd.get_dummies(data, columns=['state', 'international plan', 'voice mail plan'])

KeyError: "None of [Index(['state', 'international plan', 'voice mail plan'], dtype='object')] are in the [columns]"

In [15]:
data.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_no,international plan_yes,voice mail plan_no,voice mail plan_yes
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,1,0,0,1
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,1,0,0,1
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,1,0,1,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,1,1,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,0,1,1,0


In [16]:
data.to_excel("Data_Output.xlsx", index=False)

In [17]:
data = data.drop(columns=['voice mail plan_no', 'international plan_no']) 


In [18]:
data

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,0,0,0,1
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,0,0,0,1
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,0,1,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,36,156.2,77,26.55,215.5,126,18.32,279.1,83,...,0,0,0,0,0,0,0,0,0,1
3329,68,0,231.1,57,39.29,153.4,55,13.04,191.3,123,...,0,0,0,0,0,0,1,0,0,0
3330,28,0,180.8,109,30.74,288.8,58,24.55,191.9,91,...,0,0,0,0,0,0,0,0,0,0
3331,184,0,213.8,105,36.35,159.6,84,13.57,139.2,137,...,0,0,0,0,0,0,0,0,1,0


In [25]:
# Specify the columns of interest
columns = ['account length', 'number vmail messages', 'total day minutes', 'total day calls',
           'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge',
           'total night minutes', 'total night calls', 'total night charge', 'total intl minutes',
           'total intl calls', 'total intl charge', 'customer service calls']

In [26]:
# Generate all combinations of the columns
combinations = []
for r in range(1, len(columns) + 1):
    combinations.extend(itertools.combinations(columns, r))

In [27]:
# Calculate correlation for each combination
correlations = []
for combo in combinations:
    selected_columns = ['churn'] + list(combo)
    selected_data = data[selected_columns]
    correlation = selected_data.corr().loc['churn'].drop('churn').values[0]
    correlations.append((combo, correlation))

In [28]:
# Sort the correlations in descending order
correlations.sort(key=lambda x: abs(x[1]), reverse=True)

In [30]:
df = pd.DataFrame(correlations, columns=['Columns', 'Correlation'])

In [33]:
df.head(40)

,Columns,Correlation
0,"(customer service calls,)",0.208750
1,"(total day minutes,)",0.205151
2,"(total day minutes, total day calls)",0.205151
3,"(total day minutes, total day charge)",0.205151
4,"(total day minutes, total eve minutes)",0.205151
5,"(total day minutes, total eve calls)",0.205151
6,"(total day minutes, total eve charge)",0.205151
7,"(total day minutes, total night minutes)",0.205151
8,"(total day minutes, total night calls)",0.205151
9,"(total day minutes, total night charge)",0.205151


# Extract the features (X) and the target variable (y):

In [34]:
X = data.drop("churn", axis=1)
y = data["churn"]

In [35]:
X

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,international plan_yes,voice mail plan_yes
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,0,0,0,1
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,0,0,0,1
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,0,1,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,36,156.2,77,26.55,215.5,126,18.32,279.1,83,...,0,0,0,0,0,0,0,0,0,1
3329,68,0,231.1,57,39.29,153.4,55,13.04,191.3,123,...,0,0,0,0,0,0,1,0,0,0
3330,28,0,180.8,109,30.74,288.8,58,24.55,191.9,91,...,0,0,0,0,0,0,0,0,0,0
3331,184,0,213.8,105,36.35,159.6,84,13.57,139.2,137,...,0,0,0,0,0,0,0,0,1,0


# Perform one-hot encoding on categorical features if needed:

# Split the dataset into training and testing sets:

In [51]:
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [70]:
X_train_both.shape

(2833, 68)

In [53]:
X_train.shape

(2333, 68)

In [54]:
X_test.shape

(500, 68)

In [55]:
X_val.shape

(500, 68)

# Create and train the logistic regression model:

In [59]:
# Create and fit the logistic regression model
logistic_model = LogisticRegression(solver='liblinear', random_state=42)
logistic_model.fit(X_train, y_train)

# Create and fit the dummy classifier
dummy_model = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_model.fit(X_train, y_train)

# Predict on the validation set
logistic_preds = logistic_model.predict(X_val)
dummy_preds = dummy_model.predict(X_val)

# Evaluate the models
logistic_accuracy = accuracy_score(y_val, logistic_preds)
dummy_accuracy = accuracy_score(y_val, dummy_preds)

# Print the accuracy of the models
print("Logistic Regression Accuracy:", logistic_accuracy)
print("Dummy Classifier Accuracy:", dummy_accuracy)

Logistic Regression Accuracy: 0.866
Dummy Classifier Accuracy: 0.872


In [87]:
# Fit the logistic regression model
logistic_model = sm.Logit(y_train, sm.add_constant(X_train))
logistic_results = logistic_model.fit()

# Get the coefficients and p-values
coefs_pvalues = pd.DataFrame({'Coefficient': X_train.columns, 'Value': logistic_results.params[1:], 'p-value': logistic_results.pvalues[1:]})

# Sort the coefficients and p-values based on the absolute value of coefficients
sorted_coefs = coefs_pvalues.sort_values(by='Value', key=lambda x: abs(x), ascending=False)
sorted_pvalues = coefs_pvalues.sort_values(by='p-value')


Optimization terminated successfully.
         Current function value: 0.311392
         Iterations 8


In [96]:
# Print the sorted tables
print("Sorted Coefficients:")
print(sorted_coefs)
print("------------------------")
print("Sorted p-values:")
print(sorted_pvalues)

Sorted Coefficients:
                           Coefficient      Value   p-value
total eve charge      total eve charge -31.680043  0.190968
total night charge  total night charge -23.289171  0.337314
total intl charge    total intl charge  17.521973  0.480993
total day charge      total day charge  17.379427  0.471389
total intl minutes  total intl minutes  -4.633095  0.490185
...                                ...        ...       ...
state_NH                      state_NH   0.011549  1.000000
total eve calls        total eve calls   0.001308  0.701924
account length          account length   0.000947  0.586222
total day calls        total day calls   0.000216  0.950034
total night calls    total night calls   0.000142  0.967791

[68 rows x 3 columns]
------------------------
Sorted p-values:
                                   Coefficient     Value       p-value
international plan_yes  international plan_yes  2.344857  2.551573e-36
customer service calls  customer service calls  0.51

In [100]:
sorted_coefs['Value'].sort_values(ascending=False).head(60)


total intl charge         17.521973
total day charge          17.379427
total eve minutes          2.700523
international plan_yes     2.344857
state_MT                   1.059664
total night minutes        1.051225
state_TX                   0.751334
state_SC                   0.713290
state_NJ                   0.645669
state_AR                   0.514391
state_PA                   0.512676
customer service calls     0.510468
state_ME                   0.503896
state_NY                   0.450410
state_NV                   0.427377
state_MN                   0.424073
state_CA                   0.422360
state_WA                   0.386036
state_MI                   0.374764
state_OR                   0.330682
state_MD                   0.317242
state_MS                   0.261537
state_KS                   0.125014
state_NC                   0.123884
state_GA                   0.121493
state_KY                   0.083004
number vmail messages      0.026027
state_IN                   0

In [97]:
sorted_coefs['p-value'].sort_values(ascending=True).head(60)

international plan_yes    2.551573e-36
customer service calls    2.215095e-26
total intl calls          2.384770e-03
voice mail plan_yes       1.446426e-02
total eve minutes         1.897038e-01
total eve charge          1.909685e-01
number vmail messages     2.423960e-01
total night minutes       3.358597e-01
total night charge        3.373142e-01
total day charge          4.713894e-01
total day minutes         4.731570e-01
total intl charge         4.809930e-01
total intl minutes        4.901850e-01
account length            5.862223e-01
total eve calls           7.019245e-01
total day calls           9.500342e-01
total night calls         9.677913e-01
state_ND                  9.999998e-01
state_VT                  9.999998e-01
state_IA                  9.999998e-01
state_IL                  9.999998e-01
state_MT                  9.999998e-01
state_AL                  9.999999e-01
state_VA                  9.999999e-01
state_AZ                  9.999999e-01
state_TX                 

# Evaluate the model's performance:

# Not a good data set for logistic regression as there is not a strong linear relationship between inputs and churn 